In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from scipy.optimize import minimize
from sklearn import svm
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier
from sklearn.lda import LDA
import pandas as pd
%matplotlib inline

In [4]:
df = pd.read_csv('/Users/dekedor/Downloads/train.csv', header=0)
#df.Survived.hist()
df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(float)
median_ages = np.zeros((2, 3))
median_fares = np.zeros((2, 3))
for i in range(2):
    for j in range(3):
        median_ages[i,j] = df[(df['Gender'] == i) & \
                              (df['Pclass'] == j+1)]['Age'].dropna().median()
        median_fares[i,j] = df[(df['Gender'] == i) & \
                              (df['Pclass'] == j+1)]['Fare'].dropna().median()
 
df['AgeFill'] = df['Age']
for i in range(0, 2):
    for j in range(0, 3):
        df.loc[ (df.Age.isnull()) & (df.Gender == i) & (df.Pclass == j+1),\
                'AgeFill'] = median_ages[i,j]
        
df['Cheerburg'] = np.float64(df['Embarked'] == 'C')
df['Queenstown'] = np.float64(df['Embarked'] == 'Q')
df['South Hampton'] = np.float64(df['Embarked'] == 'S')
df['hasCabin'] = np.float64(df['Cabin'] == df['Cabin'])
df['PassengerId'] = np.float64(df['PassengerId'])
df['Survived'] = np.float64(df['Survived'])
df['Pclass'] = np.float64(df['Pclass'])
df['SibSp'] = np.float64(df['SibSp'])
df['Parch'] = np.float64(df['Parch'])
df = df.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age', 'PassengerId'], axis=1) 
m = np.shape(df['Survived'])
train_data = np.reshape(df['Survived'], (max(m)), min(m)) 
for key in df.keys():
    if not key == 'Survived':
        train_data = np.vstack((train_data, df[key]))
train_data = np.transpose(train_data)

In [5]:
forest = RandomForestClassifier(n_estimators = 100)
#m = np.shape(train_data)
#train_data = np.reshape(train_data, (max(m), min(m)))
forest = forest.fit(train_data[:, 1:], train_data[:, 0])
correct = 0
for i in range(len(train_data[:, 0])):
    correct = correct + np.int64(forest.predict(train_data[i][1:]) == train_data[i][0])

float(correct)/len(train_data[:, 0])

0.9854096520763187

In [6]:
lr = LR(C=100)
lr = lr.fit(train_data[:, 1:], train_data[:, 0])
correct = 0
for i in range(len(train_data[:, 0])):
    correct = correct + np.int64(lr.predict(train_data[i][1:]) == train_data[i][0])

float(correct)/len(train_data[:, 0])

0.813692480359147

In [7]:
df_test = pd.read_csv('/Users/dekedor/Downloads/test.csv', header=0)

df_test['Gender'] = df_test['Sex'].map( {'female': 0, 'male': 1} ).astype(float)
df_test['AgeFill'] = df_test['Age']
df_test['FareFill'] = df_test['Fare']
for i in range(0, 2):
    for j in range(0, 3):
        df_test.loc[ (df_test.Age.isnull()) & (df_test.Gender == i) & (df_test.Pclass == j+1),\
                'AgeFill'] = median_ages[i,j]
        df_test.loc[ (df_test.Fare.isnull()) & (df_test.Gender == i) & (df_test.Pclass == j+1),\
                'FareFill'] = median_ages[i,j]

df_test['Cheerburg'] = np.float64(df_test['Embarked'] == 'C')
df_test['Queenstown'] = np.float64(df_test['Embarked'] == 'Q')
df_test['South Hampton'] = np.float64(df_test['Embarked'] == 'S')
df_test['hasCabin'] = np.float64(df_test['Cabin'] == df_test['Cabin'])
df_test['PassengerId'] = np.float64(df_test['PassengerId'])
df_test['Pclass'] = np.float64(df_test['Pclass'])
df_test['SibSp'] = np.float64(df_test['SibSp'])
df_test['Parch'] = np.float64(df_test['Parch'])
passengerID = df_test['PassengerId']
df_test = df_test.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age', 'PassengerId', 'Fare'], axis=1) 

test_data = df_test['Pclass']
for key in df_test.keys():
    if not key == 'Pclass':
        test_data = np.vstack((test_data, df_test[key]))

In [8]:
m = np.shape(test_data)
test_data = np.reshape(test_data, (max(m), min(m)))
test_prediction = np.zeros(max(m))
for i in range(max(m)):
    test_prediction[i] = np.int64(forest.predict(test_data[i][:]))
test_prediction = np.vstack((passengerID, test_prediction))
test_prediction = np.transpose(test_prediction)
test_prediction = np.int64(test_prediction)

In [9]:
np.savetxt('/Users/dekedor/Downloads/predictions.csv', test_prediction, delimiter=',' \
           , header='PassengerId,Survived', fmt = '%d')

In [11]:
np.shape(train_data)
891*.6

534.6